# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

<div class="span5 alert alert-info">
### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
</div>
****

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import seaborn as sns
import statsmodels as sm
import math
%matplotlib inline


In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [3]:
print(data.shape)
black = data[data.race=='b'].reset_index(drop=True)
white = data[data.race=='w'].reset_index(drop=True)
#sns.distplot(black)
print("blacks ",black.shape)
print("whites ",white.shape)
data.head()


(4870, 65)
blacks  (2435, 65)
whites  (2435, 65)


,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


##### What test is appropriate for this problem? Does CLT apply?

 **CLT Conditions**
 
 Independent Samples Test
 
● **“Randomization”**: Each sample should represent a random sample from the population, or at least follow the population distribution.**Yes **the sample has been generated by "a randomized field experiment".

● **“10% Rule”**: The sample size must not be bigger than 10% of the entire population. 
2,435 black-sounding names and 2,435 white-sounding names represent much less than 10% of the respective race representation in the US.**Yes**

Large Enough Sample Size 

● **Sample size n should be large enough so that np≥10 and nq≥10**

each sample should have at least 10 sucessess and 10 failures.**Yes**


##### What are the null and alternate hypotheses?

H0: $M$b calls - $M$w calls = 0

H1: $M$b calls - $M$w calls ≠ 0

According to NH race has no impact on the rate of callbacks(both are same), but in contrast alternative says there is an impact on the rate of callbacks.

##### Compute margin of error, confidence interval, and p-value

Find the mean difference between two groups next, simulate many tries/samples by assuming the null hypothesis - that race has no impact. We will randomly split the population and compute the test statistic for these spits.


In [4]:
def compute_sample_stat(dist_a, dist_b):
    return abs(dist_a.mean() - dist_b.mean())

def run_simulation(df, test_stat_func):
    size = len(df)/2
    shuff = df.values
    np.random.shuffle(df.values)
    sample_a, sample_b = shuff[: size], shuff[size:]
    return test_stat_func(sample_a, sample_b)
    
def drive_simulation(iters=10000):
    diff_means = []
    for i in range(iters):
        diff_means.append(run_simulation(data.call, compute_sample_stat))
    return np.asarray(diff_means)

def compute_pval(real_mean, sample_means):
    return sum(sample_means >= real_mean)/float(len(sample_means))

In [5]:
real_mean_diff = compute_sample_stat(black.call, white.call)
print( 'Actual difference between means: {}'.format(real_mean_diff))

sample_means = drive_simulation()
p_val = compute_pval(real_mean_diff, sample_means)
std = sample_means.std()
m_nobs, m_minmax, m_mean, m_var, m_skew, m_kurt = stats.describe(sample_means)
m_std = math.sqrt(m_var)
conf_int = stats.norm.interval(0.95, loc=m_mean, scale=m_std)
print('Real Mean Difference: {}'.format(real_mean_diff))
print('Confidence Interval: {}'.format(conf_int))
print('Standard Error: {}'.format(m_std))
print('P-Value: {}'.format(p_val))


Actual difference between means: 0.03203285485506058


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:8: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


Real Mean Difference: 0.03203285485506058
Confidence Interval: (-0.0029256194359829381, 0.015345659991281266)
Standard Error: 0.004661126319510391
P-Value: 0.0


Very low P value shows that we have very strong evidence against NullHypothesis hence ** Reject H0** and ** accept the H1**.

##### Write a story describing the statistical significance in the context or the original problem

Given the p-value, test statistic, and 95% confidence interval, there is a statitistically significant difference between the number of calls a black sounding name receives and that of a white sounding name. We are 95% confident that white sounding names receive more between 0.002 and 0.015 more calls than black sounding names.


##### Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

In this analysis we can say that race is a factor in callback success but there may be other factors as well like education,experience etc. which we didn't dig in.Hence, we can conclude that with respective to this hypothesis race/name is important factor in callback success.
